# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
#Se eliminan todas las filas con al menos un valor nulo en las columnas
initial_data=spaceship.shape[0]
spaceship_1 = spaceship.dropna()
datos_totales=initial_data-spaceship_1.shape[0]
datos_totales
print(f"Se mantuvo un total de {datos_totales} datos, es decir, el {round(datos_totales/initial_data*100,2)} % del total inicial")
# Extraer la primera letra de la columna "Cabin"
spaceship_1.loc[:, 'Cabin'] = spaceship_1['Cabin'].str.strip().str[0]
#Se eliminan las columnas passengerID y name
spaceship_1= spaceship_1.drop(columns=["PassengerId", "Name"])


Se mantuvo un total de 2087 datos, es decir, el 24.01 % del total inicial


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [4]:
# One-hot encoding for sex and title
spaceship_1= pd.get_dummies(spaceship_1, columns=['HomePlanet','Cabin','Destination']) #las binarias como VIP y Cryo no se incluyen
spaceship_1.sample(4)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,...,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
7842,True,61.0,False,0.0,0.0,0.0,0.0,0.0,True,False,...,False,False,False,False,True,False,False,False,False,True
6941,False,20.0,False,0.0,802.0,0.0,16.0,0.0,False,True,...,False,False,False,False,False,True,False,False,False,True
1897,False,19.0,False,30.0,1.0,1051.0,0.0,6.0,True,True,...,False,False,False,False,True,False,False,True,False,False
3749,True,33.0,False,0.0,0.0,0.0,0.0,0.0,True,False,...,False,False,False,False,True,False,False,False,False,True


#### Quick EDA

In [5]:
import plotly.express as px
import plotly.graph_objects as go

# Se genera la matriz de correlación usando Spearman
correlation_matrix = spaceship_1.corr(method='spearman')

# Crea el gráfico de correlación utilizando plotly.graph_objects
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu',
    zmin=-1, zmax=1,
    colorbar=dict(title='Correlación', tickvals=[-1, 0, 1], ticktext=['-1', '0', '1']),
))

# Ajustar título y mostrar el gráfico
fig.update_layout(
    title='Matriz de Correlación Spearman',
    xaxis_title='Variables',
    yaxis_title='Variables',
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    width=800, height=700
)

fig.show()

**Perform Train Test Split**

In [6]:
#Se generan dataframes para características y targets
features = spaceship_1.drop(columns = ["Transported"])
target = spaceship_1["Transported"]

#Se ejecuta la división de datos para entrenar y probar el modelo, considerando X para features (variables independientes)
#  y Y para target (variable dependiente a predecir). Se asigna el 80% de los datos para entrenar el modelo, 
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=42)

Considerando que las variables Destination, Homeplanet y cabin están relacionadas entre sí, se aplica un PCA, y se generan nuevas columnas

In [7]:
# PCA para reducir la dimensión de Homeplanet 
# Se seleccionan las columnas relacionadas con el planeta de origen: 'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars'
columns=['HomePlanet_Earth', 'HomePlanet_Europa','HomePlanet_Mars']

# Se importa la clase PCA desde sklearn.decomposition para realizar la reducción de dimensionalidad
from sklearn.decomposition import PCA

# Se crea una instancia de PCA y se configura para que mantenga el 70% de la varianza original
pca_train_planet = PCA(n_components=0.7)

# Se ajusta el modelo PCA a los datos de entrenamiento. Esto calcula los componentes principales.
pca_train_planet.fit(X_train[columns])

# Se transforman los datos de entrenamiento en el nuevo espacio de componentes principales
# Esto reduce la dimensionalidad de los datos manteniendo el 70% de la varianza.
homeplanetPCA_result_train = pca_train_planet.transform(X_train[columns])

# Se obtiene el número de componentes generados en la transformación
num_components = homeplanetPCA_result_train.shape[1]

# Crear una lista de nombres de columnas para los nuevos componentes principales
# Los nombres se generan dinámicamente basados en el número de componentes generados
pca_column_names = [f'PCA_homePlanet_{i+1}' for i in range(num_components)]

# Se agregan las nuevas columnas al DataFrame X_train, las cuales contienen los componentes principales
X_train[pca_column_names] = homeplanetPCA_result_train

# Se realiza la transformación de los datos de prueba con el modelo PCA previamente ajustado
# Esto proyecta los datos de prueba en el mismo espacio de componentes principales
homeplanetPCA_result_test = pca_train_planet.transform(X_test[columns])

# Se obtiene el número de componentes generados para los datos de prueba
num_components = homeplanetPCA_result_test.shape[1]

# Crear una lista de nombres de columnas para los componentes principales de los datos de prueba
# Los nombres se generan de la misma manera que para los datos de entrenamiento
pca_column_names = [f'PCA_homePlanet_{i+1}' for i in range(num_components)]

# Se agregan las nuevas columnas al DataFrame X_test, las cuales contienen los componentes principales para los datos de prueba
X_test[pca_column_names] = homeplanetPCA_result_test


In [8]:
# PCA para reducir la dimensión de cabin
columns=['Cabin_A','Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G','Cabin_T']

# Se importa la clase PCA desde sklearn.decomposition para realizar la reducción de dimensionalidad
from sklearn.decomposition import PCA

# Se crea una instancia de PCA y se configura para que mantenga el 70% de la varianza original
pca_train_cabin = PCA(n_components=0.7)

# Se ajusta el modelo PCA a los datos de entrenamiento. Esto calcula los componentes principales.
pca_train_cabin.fit(X_train[columns])

# Se transforman los datos de entrenamiento en el nuevo espacio de componentes principales
# Esto reduce la dimensionalidad de los datos manteniendo el 70% de la varianza.
cabinPCA_result_train = pca_train_cabin.transform(X_train[columns])

# Se obtiene el número de componentes generados en la transformación
num_components = cabinPCA_result_train.shape[1]

# Crear una lista de nombres de columnas para los nuevos componentes principales
# Los nombres se generan dinámicamente basados en el número de componentes generados
pca_column_names = [f'PCA_cabin_{i+1}' for i in range(num_components)]

# Se agregan las nuevas columnas al DataFrame X_train, las cuales contienen los componentes principales
X_train[pca_column_names] = cabinPCA_result_train

# Se realiza la transformación de los datos de prueba con el modelo PCA previamente ajustado
# Esto proyecta los datos de prueba en el mismo espacio de componentes principales
cabinPCA_result_test = pca_train_cabin.transform(X_test[columns])

# Se obtiene el número de componentes generados para los datos de prueba
num_components = cabinPCA_result_test.shape[1]

# Crear una lista de nombres de columnas para los componentes principales de los datos de prueba
# Los nombres se generan de la misma manera que para los datos de entrenamiento
pca_column_names = [f'PCA_cabin_{i+1}' for i in range(num_components)]

# Se agregan las nuevas columnas al DataFrame X_test, las cuales contienen los componentes principales para los datos de prueba
X_test[pca_column_names] = cabinPCA_result_test


In [9]:
# PCA para reducir la dimensión de destination
columns=['Destination_55 Cancri e', 'Destination_PSO J318.5-22','Destination_TRAPPIST-1e']

# Se importa la clase PCA desde sklearn.decomposition para realizar la reducción de dimensionalidad
from sklearn.decomposition import PCA

# Se crea una instancia de PCA y se configura para que mantenga el 70% de la varianza original
pca_train_dest = PCA(n_components=0.7)

# Se ajusta el modelo PCA a los datos de entrenamiento. Esto calcula los componentes principales.
pca_train_dest.fit(X_train[columns])

# Se transforman los datos de entrenamiento en el nuevo espacio de componentes principales
# Esto reduce la dimensionalidad de los datos manteniendo el 70% de la varianza.
destPCA_result_train = pca_train_dest.transform(X_train[columns])

# Se obtiene el número de componentes generados en la transformación
num_components = destPCA_result_train.shape[1]

# Crear una lista de nombres de columnas para los nuevos componentes principales
# Los nombres se generan dinámicamente basados en el número de componentes generados
pca_column_names = [f'PCA_dest_{i+1}' for i in range(num_components)]

# Se agregan las nuevas columnas al DataFrame X_train, las cuales contienen los componentes principales
X_train[pca_column_names] = destPCA_result_train

# Se realiza la transformación de los datos de prueba con el modelo PCA previamente ajustado
# Esto proyecta los datos de prueba en el mismo espacio de componentes principales
destPCA_result_test = pca_train_dest.transform(X_test[columns])

# Se obtiene el número de componentes generados para los datos de prueba
num_components = destPCA_result_test.shape[1]

# Crear una lista de nombres de columnas para los componentes principales de los datos de prueba
# Los nombres se generan de la misma manera que para los datos de entrenamiento
pca_column_names = [f'PCA_dest_{i+1}' for i in range(num_components)]

# Se agregan las nuevas columnas al DataFrame X_test, las cuales contienen los componentes principales para los datos de prueba
X_test[pca_column_names] = destPCA_result_test


A continuación se realiza la selección de las variables a considerar para los modelos ML

In [11]:
#Selección para x train
X_train=X_train[['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck',
                 'PCA_homePlanet_1', 'PCA_homePlanet_2', 'PCA_cabin_1', 'PCA_cabin_2','PCA_cabin_3', 'PCA_dest_1']]

#Selección para x test
X_test=X_test[['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'PCA_homePlanet_1', 'PCA_homePlanet_2', 'PCA_cabin_1', 'PCA_cabin_2',
       'PCA_cabin_3', 'PCA_dest_1']]

Finalmente, se estandarizan los datos antes de inciiar el proceso de ML

In [12]:
# scaling the data 
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler= StandardScaler()

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [49]:
from sklearn.tree import DecisionTreeClassifier  # DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier   #  BaggingClassifier
from sklearn.metrics import classification_report  
from sklearn.metrics import confusion_matrix  # Importa la matriz de confusión

# Initialize the Pasting model 
pasting_clf = BaggingClassifier(
  estimator=DecisionTreeClassifier(max_depth=50),  
  n_estimators=500,
  max_samples=1000,
  bootstrap=False  # This ensures that no bootstrap is done, which is characteristic of Pasting
)

# Train the model with standardized data
pasting_clf.fit(X_train_scaled, y_train)

# Evaluate the model's performance
pred = pasting_clf.predict(X_test_scaled)

print("Confusion Matrix:")
print(confusion_matrix(y_test, pred))  # Matriz de confusión
# Complete classification report
report = classification_report(y_test, pred)
print(report)


Confusion Matrix:
[[529 124]
 [124 545]]
              precision    recall  f1-score   support

       False       0.81      0.81      0.81       653
        True       0.81      0.81      0.81       669

    accuracy                           0.81      1322
   macro avg       0.81      0.81      0.81      1322
weighted avg       0.81      0.81      0.81      1322



- Random Forests

In [34]:
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import classification_report, accuracy_score  # Importa clasificación
from sklearn.metrics import confusion_matrix  # Importa la matriz de confusión

# Initialize the Random Forest classifier
forest = RandomForestClassifier(n_estimators=500,
                                max_depth=30)

# Train the model with normalized data
forest.fit(X_train_scaled, y_train)

# Make predictions
pred = forest.predict(X_test_scaled)

# Evaluate the model's performance
print("Accuracy:", accuracy_score(y_test, pred))  # Exactitud
print("Confusion Matrix:")
print(confusion_matrix(y_test, pred))  # Matriz de confusión
print("Classification Report:")
print(classification_report(y_test, pred))  # Informe completo de clasificación


Accuracy: 0.8071104387291982
Confusion Matrix:
[[536 117]
 [138 531]]
Classification Report:
              precision    recall  f1-score   support

       False       0.80      0.82      0.81       653
        True       0.82      0.79      0.81       669

    accuracy                           0.81      1322
   macro avg       0.81      0.81      0.81      1322
weighted avg       0.81      0.81      0.81      1322



- Gradient Boosting

In [40]:
from sklearn.ensemble import GradientBoostingClassifier  
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix  

# Initialize the GradientBoosting classifier
gb_clf = GradientBoostingClassifier(
    max_depth=30,
    n_estimators=500
)

# Train the model with normalized data
gb_clf.fit(X_train_scaled, y_train)

# Make predictions
pred = gb_clf.predict(X_test_scaled)

# Evaluate the model's performance
print("Accuracy:", accuracy_score(y_test, pred))  # Exactitud
print("Confusion Matrix:")
print(confusion_matrix(y_test, pred))  # Matriz de confusión
print("Classification Report:")
print(classification_report(y_test, pred))  # Informe completo de clasificación


Accuracy: 0.7677760968229954
Confusion Matrix:
[[482 171]
 [136 533]]
Classification Report:
              precision    recall  f1-score   support

       False       0.78      0.74      0.76       653
        True       0.76      0.80      0.78       669

    accuracy                           0.77      1322
   macro avg       0.77      0.77      0.77      1322
weighted avg       0.77      0.77      0.77      1322



- Adaptive Boosting

In [38]:
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.tree import DecisionTreeClassifier  
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix  # Métricas de clasificación

# Initialize the AdaBoost classifier with a decision tree base classifier
ada_clf = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=100),  
    n_estimators=500
)

# Train the model with normalized data
ada_clf.fit(X_train_scaled, y_train)

# Make predictions
pred = ada_clf.predict(X_test_scaled)

# Evaluate the model's performance
print("Accuracy:", accuracy_score(y_test, pred))  # Exactitud
print("Confusion Matrix:")
print(confusion_matrix(y_test, pred))  # Matriz de confusión
print("Classification Report:")
print(classification_report(y_test, pred))  # Informe completo de clasificación



c:\Users\Gerardo\anaconda33\envs\p2_en\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning:

The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.



Accuracy: 0.773071104387292
Confusion Matrix:
[[486 167]
 [133 536]]
Classification Report:
              precision    recall  f1-score   support

       False       0.79      0.74      0.76       653
        True       0.76      0.80      0.78       669

    accuracy                           0.77      1322
   macro avg       0.77      0.77      0.77      1322
weighted avg       0.77      0.77      0.77      1322



XG BOOST

In [48]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix  # Métricas de clasificación

# Create XGBoost classifier
xgb_clf = xgb.XGBClassifier(
    max_depth=20,
    n_estimators=150,
    learning_rate=0.1,  # default learning rate
    booster='gbtree',   # default booster
    objective='binary:logistic'  # for binary classification tasks (use 'multi:softmax' for multi-class)
)

# Train the model
xgb_clf.fit(X_train_scaled, y_train)

# Make predictions
pred = xgb_clf.predict(X_test_scaled)

# Evaluate the model's performance
print("Accuracy:", accuracy_score(y_test, pred))  # Exactitud
print("Confusion Matrix:")
print(confusion_matrix(y_test, pred))  # Matriz de confusión
print("Classification Report:")
print(classification_report(y_test, pred))  # Informe completo de clasificación


Accuracy: 0.8040847201210287
Confusion Matrix:
[[524 129]
 [130 539]]
Classification Report:
              precision    recall  f1-score   support

       False       0.80      0.80      0.80       653
        True       0.81      0.81      0.81       669

    accuracy                           0.80      1322
   macro avg       0.80      0.80      0.80      1322
weighted avg       0.80      0.80      0.80      1322



Which model is the best and why?

El mejor modelo para estos datos es Random forest